<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_04_Publish_Summary_of_Web_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to search the Web and ingest web pages, and then summarize the contents.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [1]:
!pip install --upgrade graphlit-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.6/236.6 kB 3.9 MB/s eta 0:00:00


Initialize Graphlit

In [2]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [3]:
from typing import List, Optional

async def create_feed(search_text: str, read_limit: int):
    if graphlit.client is None:
        return;

    input = input_types.FeedInput(
        name="Web Search",
        type=enums.FeedTypes.SEARCH,
        search=input_types.SearchFeedPropertiesInput(
            text=search_text,
            readLimit=read_limit
        )
    )

    try:
        response = await graphlit.client.create_feed(input)

        return response.create_feed.id if response.create_feed is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def is_feed_done(feed_id: str):
    if graphlit.client is None:
        return;

    response = await graphlit.client.is_feed_done(feed_id)

    return response.is_feed_done.result if response.is_feed_done is not None else None

async def summarize_feed(feed_id: str, prompt: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.summarize_contents(
            filter=input_types.ContentFilter(
                # Filter just on feed
                feeds=[input_types.EntityReferenceFilter(id=feed_id)],
            ),
            summarizations=[
                input_types.SummarizationStrategyInput(
                    type=enums.SummarizationTypes.CUSTOM,
                    prompt=prompt
                ),
            ]
        )

        return response.summarize_contents
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def publish_feeds(prompt: str, feeds: List[str]):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.publish_contents(
            name="Web Summary",
            connector=input_types.ContentPublishingConnectorInput(
               type=enums.ContentPublishingServiceTypes.TEXT,
               format=enums.ContentPublishingFormats.MARKDOWN
            ),
            publish_prompt = prompt,
            filter=input_types.ContentFilter(
                # Filter just on feeds
                feeds=[input_types.EntityReferenceFilter(id=feed_id) for feed_id in feeds],
            ),
            is_synchronous=True
        )

        return response.publish_contents.content.markdown if response.publish_contents is not None and response.publish_contents.content is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_feeds():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_feeds(is_synchronous=True)


Execute Graphlit example

In [4]:
from IPython.display import display, Markdown
import time

# Remove any existing feeds; only needed for notebook example
await delete_all_feeds()

print('Deleted all feeds.')

read_limit = 3 # how many search results to ingest from each feed

searches = [
    "Where has Monkeypox been found?",
    "How does Monkeypox affect the body?",
    "What vaccines are being developed against Monkeypox?",
]

feeds = []

for search_text in searches:
    feed_id = await create_feed(search_text, read_limit)

    if feed_id is not None:
        print(f'Created feed [{feed_id}].')

        # Wait for feed to complete, since ingestion happens asychronously
        done = False
        time.sleep(5)
        while not done:
            done = await is_feed_done(feed_id)

            if not done:
                time.sleep(2)

        print(f'Completed feed [{feed_id}].')

        feeds.append(feed_id)

prompt = """
Read the text first, all the way to the end.
Condense the text into 5 verbose paragraphs.
Be specific when referencing persons, organizations, or any other named entities.
"""

print()
print('Summarizing feeds...')

# First, we will summarize the results of each web search, independently
for feed_id in feeds:
    summarizations = await summarize_feed(feed_id, prompt)

    if summarizations is not None:
        print(f'Summarized feed [{feed_id}]:')

        for summarization in summarizations:
            if summarization is not None and summarization.items is not None:
                for item in summarization.items:
                    display(Markdown(item.text))

publish_prompt = """
Follow these steps:
Step 1: Think about a structure for an engaging AI-generated blog post, with an introduction, an in-depth discussion of 4-6 interesting topics across all provided content, and a wrap-up.
Step 2: For each topic, write 2-3 detailed paragraphs discussing it in-depth. Touch on key points for each topic which would be interesting to readers. Mention the content metadata, entities and details from the provided summaries, as appropriate in the discussion.
Step 3: Combine all topics into a lengthy blog post. Use friendly but technical conversation to write the post.
Step 4: Remove any unnecessary formatting or final notes about being AI generated.
"""

print()
print('Publishing feeds...')

# Next, we will publish a blog post based on all the content, across all the web search feeds
published_markdown = await publish_feeds(publish_prompt, feeds)

if published_markdown is not None:
    display(Markdown(published_markdown))

Deleted all feeds.
Created feed [ef3ad852-0c2b-4fd3-bba4-1600adf4ec22].
Completed feed [ef3ad852-0c2b-4fd3-bba4-1600adf4ec22].
Created feed [ecc70b22-2ce3-461c-91af-c000fc041f7a].
Completed feed [ecc70b22-2ce3-461c-91af-c000fc041f7a].
Created feed [b8c3d267-84f0-4686-b119-1a031882a4ad].
Completed feed [b8c3d267-84f0-4686-b119-1a031882a4ad].

Summarizing feeds...
Summarized feed [ef3ad852-0c2b-4fd3-bba4-1600adf4ec22]:


Mpox, formerly known as monkeypox, is a viral illness caused by the monkeypox virus, a member of the Orthopoxvirus genus. The virus has two distinct clades: clade I, with subclades Ia and Ib, and clade II, with subclades IIa and IIb. A global outbreak of mpox, primarily caused by the clade IIb strain, began in 2022 and continues to affect various regions, including Africa. The Democratic Republic of the Congo has seen a significant rise in cases due to clades Ia and Ib. Mpox is characterized by symptoms such as a skin rash, fever, headache, muscle aches, and swollen lymph nodes. Transmission occurs through close contact with infected individuals, contaminated materials, or animals. Vaccination is available and should be considered alongside other public health measures.

The transmission of mpox primarily occurs through close contact with an infected person, including skin-to-skin and respiratory interactions. People with multiple sexual partners are at a higher risk of contracting the virus. Mpox can also spread through contaminated objects, needle injuries, and from mother to child during pregnancy or birth. Animal-to-human transmission is possible through bites, scratches, or handling infected animals. The natural reservoir of the virus remains unknown, but small mammals like squirrels and monkeys are susceptible. Research is ongoing to better understand the virus's transmission dynamics in different settings.

Symptoms of mpox typically appear within a week of exposure and last 2–4 weeks. The disease manifests as a rash that can spread across the body, starting as flat sores that develop into blisters. Other symptoms include fever, sore throat, and swollen lymph nodes. Some individuals may experience severe complications, such as bacterial skin infections, pneumonia, or encephalitis. Diagnosis can be challenging due to symptom overlap with other diseases, making testing crucial. The preferred diagnostic method is PCR testing of samples from the rash. HIV testing is recommended for adults with mpox, and other diagnostic tests may be considered.

Treatment for mpox focuses on symptom management and preventing complications. Vaccination is advised for high-risk groups, including healthcare workers and individuals with multiple sexual partners. The vaccine can also be administered post-exposure to prevent infection. While some antivirals have emergency use authorization, no proven effective antiviral treatment exists for mpox. Supportive care remains a priority, and individuals with HIV should continue antiretroviral therapy. Self-care measures include isolating at home, maintaining hygiene, and avoiding contact with others until recovery. Health workers should adhere to infection prevention protocols when caring for mpox patients.

The first human case of mpox was reported in 1970 in the Democratic Republic of the Congo. Since the eradication of smallpox, mpox has emerged in various African regions. A notable outbreak in the United States in 2003 was linked to imported animals. In 2022, a global outbreak spread rapidly across Europe, the Americas, and other regions, primarily affecting men who have sex with men. The World Health Organization (WHO) has declared mpox a public health emergency of international concern twice, in May 2022 and August 2024. WHO continues to work with member states to prevent and respond to mpox outbreaks, focusing on equitable access to vaccines and treatments. Stigma and discrimination against affected communities remain significant challenges in managing the outbreak.

In May 2022, a significant outbreak of monkeypox was reported, marking an unusual spread of the virus beyond its typical confines in Africa. Historically, monkeypox cases have been largely restricted to 11 African countries, but this outbreak saw the virus identified in 16 countries outside of Africa, including a dozen in Europe, as well as the United States, Australia, Canada, and Israel. This surge in cases is notable because it represents a fourfold increase in the number of countries reporting monkeypox outside Africa compared to the previous 50 years. The outbreak has raised concerns among health experts about the potential for the virus to become more widespread.

The outbreak has resulted in at least 160 confirmed cases in non-African countries, with the majority occurring in Europe. The United Kingdom reported 56 cases, Spain 41, and Portugal 37, with smaller numbers in Austria, Belgium, Denmark, France, Germany, Italy, the Netherlands, Sweden, and Switzerland. This is a significant development, as the largest previous outbreak in the Western Hemisphere occurred in 2003, when pet prairie dogs infected 47 people in the United States. Anne Rimoin, an epidemiology professor at UCLA, highlighted the importance of this outbreak, noting it as the most significant poxvirus emergence since smallpox eradication.

The current outbreak has primarily affected young men who have sex with men, suggesting that the virus is spreading through skin-to-skin contact, possibly during sexual activity. This pattern of transmission is unusual for monkeypox, which typically spreads through contact with infected animals. The World Health Organization (WHO) has noted that many of the affected individuals have no recent travel history to Africa or known exposure to infected persons, indicating a new mode of transmission that requires further investigation.

Monkeypox is endemic in the Democratic Republic of the Congo (DRC) and Nigeria, where the virus has been present for decades. The DRC reported over 18,000 confirmed or suspected cases from 2010 to 2019, with more than 6,000 suspected cases in 2020 alone. Nigeria has reported over 200 confirmed cases and more than 500 suspected cases since 2017. The recent cases outside Africa belong to the West African clade of monkeypox, which is less severe than the Congo Basin clade, with a mortality rate of around 1% compared to 10% for the latter.

Despite the low risk to the general population, the outbreak underscores the global nature of infectious diseases. Dr. Agam Rao from the CDC emphasized that all imported cases outside Africa since 2018 have originated from Nigeria. Anne Rimoin reiterated the importance of recognizing that infections can spread globally, highlighting the need for vigilance and preparedness in managing infectious disease outbreaks. This situation serves as a reminder of the interconnectedness of global health and the potential for diseases to transcend geographical boundaries.

NBC News has been diligently tracking the spread of mpox cases across the United States, a disease that the World Health Organization (WHO) has classified as an international public health emergency. The coverage, which is updated weekly, provides a comprehensive overview of the situation as it evolves. Initially, the outbreak saw a significant surge, with cases peaking at over 400 per day in August. However, recent data indicates a substantial decline, with fewer than five cases reported daily as of early January. This decline marks a significant reduction from the earlier peak, reflecting the efforts to control the spread of the virus.

The Centers for Disease Control and Prevention (CDC) has reported that mpox cases have been documented in every U.S. state, a notable increase from the 44 states and Washington, D.C., that had reported cases by the end of July. This expansion of reported cases highlights the widespread nature of the outbreak. Since May, the United States has recorded over 29,000 cases, contributing to a global total that exceeds 80,000. The virus primarily spreads through close physical contact, with recent cases predominantly affecting men who have sex with men.

In response to the outbreak, several states have taken decisive action. The WHO's declaration of a public health emergency in July prompted states like California, Illinois, and New York to issue their own emergency declarations. These measures underscore the seriousness with which state governments are treating the outbreak and their commitment to mitigating its impact. The emergency declarations are part of broader efforts to mobilize resources and implement strategies to curb the spread of the virus.

The renaming of monkeypox to mpox by the WHO in November was a significant development, aimed at addressing concerns about racism associated with the original name. This change reflects a broader effort to ensure that public health communications are sensitive and inclusive. The renaming is part of ongoing efforts to address not only the medical aspects of the outbreak but also the social and cultural dimensions that can influence public perception and response.

The coverage of the mpox outbreak by NBC News is spearheaded by Joe Murphy, a data editor, and Alex Ford, an interactive journalist. Their work, along with contributions from others like Dana Varinsky, provides valuable insights into the dynamics of the outbreak. By analyzing CDC data and other sources, they offer a detailed picture of how the situation is unfolding, helping the public stay informed about the latest developments and the measures being taken to address the crisis.

Summarized feed [ecc70b22-2ce3-461c-91af-c000fc041f7a]:


...

Monkeypox (mpox) is a zoonotic disease caused by the monkeypox virus (MPXV), which poses significant health risks, particularly to pregnant individuals, children, and those with compromised immune systems. The World Health Organization declared a global health emergency in August 2024 due to a recent mpox outbreak, emphasizing the urgent need for effective vaccines. This review explores various vaccine platforms for mpox prevention, including attenuated and non-replicating virus vaccines, viral vector-based vaccines, recombinant protein vaccines, and DNA and mRNA vaccines. Each platform's advantages and limitations are assessed, highlighting the necessity for ongoing research to improve vaccine efficacy and safety. Recent advancements, such as immunostimulatory sequences and improved delivery systems, are discussed for their potential to enhance protection against diverse orthopoxvirus strains.

The pathophysiology of mpox involves the virus entering the human body through skin wounds, mucous membranes, or the respiratory tract, leading to replication at the entry site and subsequent spread to lymph nodes and other organs. This process results in viremia and systemic symptoms like fever and muscle aches. Lymphadenopathy is a distinctive feature of mpox, and complications are more common in children and immunocompromised individuals. The virus's genetic diversity, driven by genomic changes, underscores the importance of continuous surveillance and vaccine development. Recent vaccine candidates, including multivalent mRNA and DNA vaccines, show promise in providing strong immune responses and comprehensive disease prevention.

Current vaccines for MPXV include JYNNEOS, LC16m8, and ACAM2000. JYNNEOS, a live, non-replicating vaccine, is approved in the USA and Mexico and offers a safer option for immunocompromised individuals. LC16m8, used in Japan, is a live attenuated vaccine with a favorable safety profile. ACAM2000, a live, replicating vaccine, is effective but associated with significant side effects, limiting its use to non-pregnant, immunocompetent individuals. The development of vaccines against MPXV leverages the cross-protection between VACV and MPXV, targeting conserved viral proteins. Ongoing clinical trials aim to evaluate the safety, dosage, and efficacy of MPXV vaccines across various populations.

Emerging vaccine platforms, such as viral vector-based, recombinant protein-based, DNA-based, and mRNA vaccines, offer diverse approaches to mpox prevention. Viral vector-based vaccines utilize non-replicating viruses to induce robust immune responses, while recombinant protein-based vaccines use specific viral proteins to trigger immune responses without live pathogens. DNA vaccines introduce plasmids encoding specific antigens, offering simplicity and cost-effectiveness. mRNA vaccines, which do not interact with the genome, provide superior safety and can induce both cellular and humoral immune responses. Each platform has unique advantages and challenges, and ongoing research aims to optimize their efficacy, safety, and logistical feasibility.

Future strategies for enhancing mpox vaccine efficacy and stability include incorporating immunostimulatory sequences, improving delivery systems, and developing polyvalent vaccines. These approaches aim to provide broader protection against orthopoxvirus strains and address the limitations of current vaccines. The recent global mpox epidemic highlights the urgent need for new, effective vaccines, as first-generation vaccines present significant complications. Continued research and development are essential to ensure population protection against future outbreaks and potential bioterrorist threats. Addressing scientific, logistical, regulatory, and socioeconomic challenges is crucial for efficient immunization and global public health.

The monkeypox virus (MPOX), a zoonotic illness caused by an orthopoxvirus, has become a significant public health concern, with 87,113 confirmed cases and 111 fatalities reported across 110 countries as of April 25, 2023. The virus, which presents symptoms similar to smallpox, has seen a resurgence partly due to the cessation of smallpox vaccinations over the past four decades. The World Health Organization (WHO) has recommended the use of affordable MPOX vaccines, which include replicating (ACAM2000), low replicating (LC16m8), and non-replicating (MVA-BN) vaccines. These vaccines, originally developed for smallpox, provide cross-protection against MPOX, with smallpox vaccination being approximately 85% effective in preventing MPOX. However, the effectiveness of these vaccines in the current multi-country outbreak remains to be fully verified.

The MPOX virus is part of the orthopoxvirus genus, which includes the variola virus responsible for smallpox. The virus is primarily found in African nations, with rodents as the likely primary reservoir. Human-to-human transmission occurs mainly through close contact with symptomatic patients or infected animals. The virus has two major clades, West Africa and the Congo Basin, and has caused several significant outbreaks since 1970. The current outbreak in 2023 has highlighted the need for effective vaccines, as the virus can cause severe illness or death in immunocompromised individuals, children, the elderly, pregnant women, and those with co-morbidities. The re-emergence of MPOX is attributed to the discontinuation of smallpox vaccination, leading to waning immunity in the population.

Efforts to develop vaccines against MPOX have focused on various strategies, including inactivated, live-attenuated, virus-like particles (VLPs), recombinant protein, nucleic acid, and nanoparticle-based vaccines. Traditional vaccines, such as Dryvax and ACAM2000, have been effective but come with side effects, particularly in immunocompromised individuals. Newer vaccines like JYNNEOS, a non-replicating vaccine, offer protection with fewer side effects and are suitable for immunocompromised individuals. The development of novel vaccine platforms, such as mRNA and DNA-based vaccines, is ongoing, with the potential to provide safe and effective protection against MPOX and other orthopoxviruses.

The current status of MPOX vaccines includes several in advanced clinical development, such as IMVAMUNE, MVA-BN, and Imvanex. These vaccines are being evaluated for their safety and immunogenicity in various populations, including healthcare workers and those at high risk of exposure. The WHO has emphasized the importance of supporting vaccination programs with surveillance and contact tracing, as well as public health messaging and pharmacovigilance. The availability and safety concerns of existing vaccines like ACAM2000 and MVA are being addressed, with a focus on developing new vaccines that are safe, effective, and accessible to all populations.

In conclusion, the global spread of MPOX necessitates the development of novel vaccines with broad use potential. mRNA-based vaccines have shown promise in inducing strong immune responses and protection against MPOX. However, further studies are needed to assess the long-term safety and efficacy of these vaccines, especially in high-risk populations. The lessons learned from the COVID-19 pandemic highlight the need for rapid development and deployment of vaccines to control and contain MPOX. Advances in bio/nanotechnology and open data sharing have laid the groundwork for the development of new vaccine technologies to address the evolving threat of MPOX.

The article discusses the challenges and strategies related to the production and distribution of vaccines for monkeypox (MPX), drawing lessons from the COVID-19 pandemic. Monkeypox is a zoonotic infection caused by a virus related to smallpox, and it has recently spread to over 89 countries, with more than 79,000 cases reported globally. The virus can be transmitted through respiratory droplets, direct contact with bodily fluids, and contaminated objects, as well as through contact with infected animals. The current outbreak is notable for its rapid spread through sexual contact. Despite the severity of the situation, there is no specific medicine or vaccine available for MPX, which poses a significant challenge, especially for middle- and low-income countries already strained by the COVID-19 pandemic.

Currently, three vaccines—ACAM2000, MVA-BN (JYNNEOS), and LC16—are available for MPX, though they are not widely used due to potential adverse reactions. ACAM2000 is a live attenuated vaccine approved by the FDA for smallpox and MPX, administered via a scarification technique. MVA-BN, also FDA-approved, is a non-replicating vaccine given in two doses. LC16, manufactured by KM Biologics, is not FDA-approved for MPX. The Indian government has issued guidelines for managing MPX, emphasizing monitoring and isolation of contacts. However, the lack of a specific MPX vaccine highlights the need for new vaccine development, considering the potential for cross-reactivity with smallpox vaccines.

The article highlights the challenges in vaccine production and distribution, particularly the need for a specific MPX vaccine. The re-emergence of MPXV requires addressing new immune evasion mechanisms. The cessation of smallpox vaccination has left a large portion of the population unvaccinated, complicating the rapid production of smallpox vaccines. Additionally, logistical issues such as storage, transportation, and distribution pose significant challenges, especially in remote areas. India's vaccine production capacity is substantial, but the country faces hurdles in vaccine delivery due to decentralized public health systems and vaccine hesitancy, as seen during the COVID-19 pandemic.

The article emphasizes the importance of international collaboration to overcome challenges during public health emergencies. High-income countries often stockpile vaccines, creating shortages in low- and middle-income countries. The COVID-19 pandemic demonstrated the need for global cooperation rather than competition in vaccine distribution. The article suggests that high-income countries should work with lower-income countries to ensure fair distribution and management of resources. Vaccine donors and pharmaceutical companies should collaborate with healthcare workers and researchers to develop a coordinated response plan.

In conclusion, the article recommends that infected individuals follow guidelines to prevent the spread of MPX, such as wearing masks and isolating until lesions heal. The WHO advises against mass vaccination, instead recommending active surveillance and isolation. Research into new vaccines using advanced molecular techniques is encouraged to prevent future shortages. International collaboration and support are crucial for effective vaccine manufacturing and distribution. The article underscores the need for a coordinated global response to tackle the MPX health emergency effectively.


Publishing feeds...


# Understanding Mpox: A Comprehensive Overview

Mpox, formerly known as monkeypox, has emerged as a significant public health concern in recent years. This zoonotic viral disease, caused by the monkeypox virus (MPXV), shares similarities with smallpox and has been the subject of global attention due to its outbreaks across various continents. In this blog post, we delve into the intricacies of mpox, exploring its transmission, symptoms, global spread, vaccine development, and the challenges faced in managing this infectious disease.

## The Nature of Mpox and Its Transmission

Mpox is a viral illness caused by the monkeypox virus, a member of the Orthopoxvirus genus. It is primarily a zoonotic disease, meaning it can be transmitted from animals to humans. The virus is known to spread through direct contact with infected animals, such as rodents and nonhuman primates, as well as through human-to-human transmission via respiratory droplets, bodily fluids, and contaminated objects. Notably, the 2022 outbreak highlighted the virus's ability to spread through close physical contact, particularly among men who have sex with men, emphasizing the need for targeted public health interventions.

The symptoms of mpox are similar to those of smallpox, including fever, headache, muscle aches, and a distinctive rash that progresses through several stages. The rash often begins on the face and spreads to other parts of the body, including the palms, soles, and genitalia. While most cases are mild and self-limiting, complications such as pneumonia, encephalitis, and secondary infections can occur, particularly in immunocompromised individuals and pregnant women.

## Global Spread and Public Health Response

Historically, mpox was endemic to Central and West Africa, with sporadic cases reported in other regions. However, the 2022 outbreak marked a significant shift, with cases reported in over 89 countries, including the United States, Europe, and Australia. This unprecedented spread prompted the World Health Organization (WHO) to declare mpox a public health emergency of international concern, underscoring the need for coordinated global efforts to control the virus.

The outbreak has been closely monitored by health organizations worldwide, with the Centers for Disease Control and Prevention (CDC) and WHO playing pivotal roles in tracking cases and providing guidance. Public health measures, including vaccination campaigns, isolation of infected individuals, and contact tracing, have been implemented to curb the spread of the virus. The renaming of monkeypox to mpox by the WHO in November 2022 aimed to reduce stigma and ensure that disease names do not perpetuate discrimination.

## Vaccine Development and Challenges

The development of effective vaccines against mpox has been a critical focus in the fight against the virus. Current vaccines, such as JYNNEOS and ACAM2000, provide cross-protection against mpox, but their specific effectiveness is still under evaluation. These vaccines are primarily used for smallpox prevention and have been repurposed for mpox due to the genetic similarities between the viruses. However, challenges remain in ensuring vaccine availability, affordability, and distribution, particularly in low- and middle-income countries.

Recent advancements in vaccine technology have introduced novel platforms, including mRNA and DNA vaccines, which offer promising alternatives to traditional vaccines. These platforms have the potential to provide broader protection against diverse orthopoxvirus strains and are being explored in ongoing clinical trials. The success of mRNA vaccines during the COVID-19 pandemic has spurred interest in their application for mpox, highlighting the need for continued research and innovation in vaccine development.

## Lessons from the COVID-19 Pandemic

The COVID-19 pandemic has provided valuable lessons in managing infectious disease outbreaks, particularly in terms of vaccine distribution and public health preparedness. The rapid development and deployment of COVID-19 vaccines demonstrated the potential of nucleic acid platforms and the importance of international collaboration in addressing global health challenges. These lessons are being applied to the mpox outbreak, with efforts focused on ensuring equitable vaccine distribution and enhancing surveillance and response mechanisms.

Public acceptance and education efforts are crucial in countering misinformation and building trust in vaccination campaigns. Effective communication strategies and robust safety monitoring systems are essential components of successful immunization efforts, particularly in resource-limited settings. The mpox outbreak serves as a reminder of the interconnectedness of global health systems and the need for coordinated responses to prevent future pandemics.

## Future Directions and Global Preparedness

As the world continues to grapple with the challenges posed by mpox, ongoing research and development of new vaccines remain a priority. The integration of scientific advancements, such as immunoinformatics and reverse vaccinology, promises to revolutionize vaccine design and enhance immune responses. Collaborative efforts between governments, industry, and academia are vital in advancing vaccine research and ensuring access to vaccines for high-risk populations.

The mpox outbreak underscores the importance of global health security and the need for robust surveillance and response strategies. By learning from past experiences and leveraging technological innovations, the global community can better prepare for and mitigate the impact of future infectious disease outbreaks.